# **Практическое занятие 14. Случайный лес (часть 3)**

Котпев Данил

Построить  модель  Случайный  лес  для  решения  задачи  прогнозирования  цены  дома, используя данные набора House Prices Competition.

**Подключим библиотеки**

In [161]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

**Задание 4**

1. Создать и обучить на обучающей выборке случайный лес с параметрами по умолчанию. Оценить качество полученной модели на тестовых данных.

In [162]:
dataAll = pd.read_csv("house_prices.csv")

# Выбор нужных признаков
dataNum = ["1stFlrSF", "OverallQual", "GrLivArea", "GarageArea", "YearBuilt", "FullBath"]
dataCat = ["GarageFinish",  "Neighborhood", "KitchenQual", "GarageType", "ExterQual"]
data = dataAll[dataNum + dataCat]

# Разделение данных на обучающую и тестовую выборки
salePrice = dataAll["SalePrice"]
x_train, x_test, y_train, y_test = train_test_split(data, salePrice, test_size=0.3, train_size=0.7, random_state=42)

In [163]:
pipelineDataNum = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')), #Заменяем пропущенные значения медианной (лучше MAE)
        ('scaler', StandardScaler()) #Для масштабирования используем стандратизацию
        ])

# Реализуем бинаризацию для качественных признаков
pipelineDataCat = Pipeline([
    ('encoder', OneHotEncoder(handle_unknown='ignore')),
    ('imputer', SimpleImputer(strategy='median')), #Заменяем пропущенные значения медианной (лучше MAE)
])

preprocessor = ColumnTransformer(
    transformers=[
        ('quant', pipelineDataNum , dataNum),
        ('qual', pipelineDataCat, dataCat)])

pipeOrigin = Pipeline([("preprocessing", preprocessor), ("regressor", RandomForestRegressor() )])

pipeOrigin.fit(x_train, y_train)

predOrigin = pipeOrigin.predict(x_test)

print(f"MAE:", mean_absolute_error(y_test, predOrigin))

MAE: 18402.01794661883


2. Поэкспериментировать с гиперпараметрами модели: количество базовых алгоритмов (деревьев)  и  максимальная  глубина  деревьев;  оценить  качество  всех  полученных моделей.

In [164]:
# Создание модели случайного леса
rf = RandomForestRegressor()

# Создание сетки параметров для перебора
param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': np.arange(5, 21, 5)
}

# Перебор параметров с помощью RandomizedSearchCV
grid_search = RandomizedSearchCV(rf, param_dist, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(preprocessor.fit_transform(x_train), y_train)

# Вывод лучших параметров и score на экран
print("Лучшие параметры:", grid_search.best_params_)
print("Score на обучающей выборке:", grid_search.best_score_)

# Оценка качества модели на тестовой выборке
rf_best = grid_search.best_estimator_
pred = rf_best.predict(preprocessor.transform(x_test))
mae = mean_absolute_error(y_test, pred)
print("MAE на тестовой выборке:", mae)

Лучшие параметры: {'n_estimators': 150, 'max_depth': 15}
Score на обучающей выборке: -1351930048.518417
MAE на тестовой выборке: 18417.547353436374


3. Проанализировать результаты проведенных экспериментов, сделать выводы о влиянии значений гиперпарметров на предсказательную способность модели.
4. Выбрать лучшую модель (по результатам выполнения п. 2); сравнить предсказательную способность этой модели и модели линейной регрессии, полученной при выполнении предыдущих заданий.
5. По результатам проведенного исследования оформить отчет.

**Отчет**

  3. После проведения эксперемента  была определена лучшая модель на обучающей выборке с 150 деревьями решений и максимальной глубиной деревьев 15. 

  Чем больше количество деревьев, тем более сложная модель, но тем лучше она справляется с задачей регрессии. Однако, при увеличении количества деревьев, модель может стать более сложной и потребовать больше времени на обучение и предсказание, но не всегда все зависит от происхождения данных. Аналогичное можно сказать и про глубину дерева.

  4. После проведения эксперемента  была определена лучшая модель на обучающей выборке с 150 деревьями решений и максимальной глубиной деревьев 15 c MAE на тестовой выборке равной 18267.755536830053.

  Лучшие результаты по выполнению прошлых работ стали c использованием линейной регресси:

  * Лучшей моделью L1 является с коэфицентом регуляризатора 1 с МАЕ равной 21723.38440636757

  * Лучшей моделью L2 является с коэфицентом регуляризатора 1 с МАЕ равной 21604.258826764028

  Сравнивая данные можно с уверенностью сказать, что случайный лес подходит лучше для решения данной задачи. Скорее потому что случайный лес может обрабатывать нелинейные зависимости между признаками и значениями целевой переменной, что делает его более гибким и способным адаптироваться к различным типам данных.

**Задание 5**

Построить модель Случайный лес для решения задачи кредитного скоринга, используя данные клиентов банка из файла scoring.csv

1. Создать и обучить на обучающей выборке случайный лес с параметрами по умолчанию. Оценить качество полученной модели на тестовых данных.

In [166]:
dataAll = pd.read_csv('scoring.csv', sep = ';')
dataAll = pd.get_dummies(dataAll, columns=['purpose'])
dataAll.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   account                      600 non-null    int64  
 1   duration                     600 non-null    float64
 2   credit_history               600 non-null    int64  
 3   amount                       600 non-null    float64
 4   savings                      600 non-null    int64  
 5   employment                   600 non-null    int64  
 6   guarantors                   600 non-null    int64  
 7   residence                    600 non-null    int64  
 8   age                          600 non-null    float64
 9   credits_number               600 non-null    int64  
 10  job                          600 non-null    int64  
 11  maintenance_people           600 non-null    int64  
 12  telephone                    600 non-null    int64  
 13  foreign             

In [167]:
data = dataAll.drop(['target'], axis=1)
target = dataAll['target']

scaler = MinMaxScaler()
scaler.fit(data)
data_scaled = scaler.transform(data)

x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.3, train_size=0.7, random_state=42)

In [168]:
model = RandomForestClassifier(random_state = 42)
model.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [169]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

In [170]:
# получаем предсказания для тестовой и обучающейся выборке
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# вычисление метрики accuracy
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)
# вычисление метрики MAE
mae_train = mean_absolute_error(y_train, y_train_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f"Accuracy на обучающейся выборке: {accuracy_train:.4f}")
print(f"Accuracy на тестовой выборке: {accuracy_test:.4f}")

Accuracy на обучающейся выборке: 1.0000
Accuracy на тестовой выборке: 0.7056


2. Поэкспериментировать с гиперпараметрами модели: количество базовых алгоритмов (деревьев),  максимальная  глубина  деревьев,  критерий  информативности;  оценить качество всех полученных моделей.

In [171]:
# Перебор параметров с помощью RandomizedSearchCV
grid_search = RandomizedSearchCV(model, param_dist, scoring = 'accuracy', random_state = 42)
grid_search.fit(x_train, y_train)

# Вывод лучших параметров и score на экран
print("Лучшие параметры:", grid_search.best_params_)
print("Score на обучающей выборке:", grid_search.best_score_)

# Оценка качества модели на тестовой выборке
rf_best = grid_search.best_estimator_

y_test_pred = rf_best.predict(x_test)
y_train_pred = rf_best.predict(x_train)

Лучшие параметры: {'n_estimators': 50, 'max_depth': 10}
Score на обучающей выборке: 0.719047619047619


In [172]:
# вычисление метрики accuracy
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print(f"Accuracy на обучающейся выборке: {accuracy_train:.4f}")
print(f"Accuracy на тестовой выборке: {accuracy_test:.4f}")

Accuracy на обучающейся выборке: 0.9929
Accuracy на тестовой выборке: 0.7222


3. Проанализировать результаты проведенных экспериментов, сделать выводы о влиянии значений гиперпарметров на предсказательную способность модели.

4. Выбрать лучшую модель (по результатам выполнения п. 2); сравнить предсказательную способность  этой  модели  и  модели  логистической  регрессии,  полученной  при выполнении соответствующего задания.

5. По результатам проведенного исследования оформить отчет.

**Отчет**

  3. После проведения эксперемента  была определена лучшая модель на обучающей выборке с 50 деревьями решений и максимальной глубиной деревьев 10. 

  Чем больше количество деревьев, тем более сложная модель, но тем лучше она справляется с задачей регрессии. Однако, при увеличении количества деревьев, модель может стать более сложной и потребовать больше времени на обучение и предсказание, но не всегда все зависит от происхождения данных. Аналогичное можно сказать и про глубину дерева.

  4. После проведения эксперемента  была определена лучшая модель на обучающей выборке с 50 деревьями решений и максимальной глубиной деревьев 10 c accuracy на тестовой выборке равной 0.7222.

  Лучшие результаты по выполнению прошлых работ стали c использованием логистической регресси:

 * Точность на обучающей выборке: 0.7238095238095238
 * Точность на тестовой выборке: 0.6833333333333333 

  Из результатов, полученных в предыдущих работ, видно, что модель случайного леса имеет более высокую точность на обучающей выборке и на тестовой выборке, чем модель логистической регрессии.

  Это может быть связано с тем, что модель случайного леса может обрабатывать нелинейные зависимости между признаками и целевой переменной, что позволяет ей лучше поглощать шум и широкий диапазон значений целевой переменной.